In [ ]:

#Install required libraries
import pandas as pd
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.query import Gen3Query

In [ ]:
### download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
# import os
#os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py")


In [ ]:
from stress_testing.expansion_bak import Gen3Expansion

In [ ]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
api = 'https://qa-midrc.planx-pla.net/'
cred = 'midrc_api_key.json'
auth = Gen3Auth(api, refresh_file=cred) # authentication class
# print(auth.get_access_token())
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()

In [ ]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
api = 'https://piotr.planx-pla.net/'
cred = 'piotr_api_key.json'
auth = Gen3Auth(api, refresh_file=cred) # authentication class
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()

In [ ]:
img_stud = exp.get_node_tsvs(projects = 'jnkns-jenkins', node='imaging_study', overwrite=True)

In [ ]:
# #increasing the volume of the number of records by 10 fold for testing purposes
# img_stud_list = [img_stud.copy(deep=True) for i in range(10)]
# for i in range(10):
#     img_stud_list[i]["submitter_id"] = img_stud['submitter_id']+"_"+str(i) #changing the submitter_id to make them unique

# img_stud = pd.concat(img_stud_list)


type
submitter_id	
datasets
cases

In [ ]:
# Trim down the number of records to 4000 to save time on submission
img_stud = img_stud.head(4000)
len(img_stud)

In [ ]:
img_stud = img_stud[['type', 'submitter_id', 'datasets.submitter_id', 'cases.submitter_id']]

img_stud['datasets.submitter_id'] = 'dataset_backpedalled_usurers'

img_stud.to_csv('test_img_stud.tsv', sep='\t', index=False)



In [ ]:
import time
start = time.time()
sub_test = sub.submit_file(project_id = 'jnkns-jenkins', filename = 'test_img_stud.tsv', chunk_size=1000)
end = time.time()
time_taken = end - start


In [ ]:

print(f"time taken to push {len(img_stud)} recoords with sheepdog piotr_laod_testing workers = 2 and psycopg2-binary = 3.9.0 - {time_taken}")

#These are the values of the time taken to push 8100 records to the service
time_load_test_branch = 628.9082880020142  #But the chunk size was cut down to 500
time_load_test_branch_new_base = 641.9752421379089 #but service didn't fail after 1min, keeping chunk size at 1000
time_load_2501 = 299.92830085754395 # Benchnark time for 8100 records with tag 25.01


In [ ]:
print("time taken to push 4000 recoords with sheepdog load_test_branch with (num_procs * 2) + 1 workers- 408.41149735450745")
print("time taken to push 4000 recoords with sheepdog load_test_branch with workers = 1 --> 402.67703795433044")
print("time taken to push 4000 recoords with sheepdog 2025.01 - 161.08299779891968")
print("time taken to push 4000 recoords with sheepdog load_testing_branch with preload_app = False and 4 workers- 371.7728190422058")
print("time taken to push 4000 recoords with sheepdog load_testing workers = 1, preload = True - 427.6445198059082")
print("time taken to push 4000 recoords with sheepdog piotr_laod_testing workers = 4,  - 316.553505897522")
print("time taken to push 4000 recoords with sheepdog piotr_laod_testing workers = 2 - 369.94946694374084")